## Tool Calling with Agentic AI - AutoGen

### LLM Used - Granite3.1-8B

In this notebook we will learn how to use Tool Calling with Agentic AI in order to solve different problems.

Tool-calling agents expand the capabilities of an LLM by allowing it to interact with external systems. This approach empowers agents to dynamically solve problems by utilizing tools, accessing memory, and planning multi-step actions.

Tool calling agents enable:

1. Multi-Step Decision Making: The LLM can orchestrate a sequence of decisions to achieve complex objectives.
2. Tool Access: The LLM can select and use various tools as needed to interact with external systems and APIs.

This architecture allows for more dynamic and flexible behaviors, enabling agents to solve complex tasks by leveraging external resources efficiently.

In [12]:
!pip install -q autogen==0.7.5 autogen-agentchat~=0.2 psutil==5.9.8


[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import autogen
from typing import Literal

from pydantic import BaseModel, Field
from typing_extensions import Annotated

import autogen
from autogen.cache import Cache

INFERENCE_SERVER_URL = os.getenv('API_URL_GRANITE')
MODEL_NAME = "granite-3-8b-instruct"
API_KEY= os.getenv('API_KEY_GRANITE')

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


In [3]:
# Configuration for the vLLM endpoint
local_llm_config = {
    "config_list": [
        {
            "model": MODEL_NAME,
            "api_key": API_KEY, 
            "base_url": f"{INFERENCE_SERVER_URL}/v1"
        }
    ],
    "cache_seed": None,
    "temperature": 0.01,
    "timeout": 600,
}

In [4]:
from autogen import ConversableAgent

agent = ConversableAgent(
    "chatbot",
    llm_config=local_llm_config,
    code_execution_config=False,  # Turn off code execution, by default it is off.
    function_map=None,  # No registered functions, by default it is None.
    human_input_mode="NEVER",  # Never ask for human input.
)

reply = agent.generate_reply(messages=[{"content": "What is Agentic AI?", "role": "user"}])
print(reply)

[autogen.oai.client: 12-05 12:57:01] {351} WARNING - Model granite30-8b is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
{'content': 'Agentic AI refers to artificial intelligence systems that are designed to act autonomously and make decisions on their own, rather than being controlled by humans. These systems are often equipped with advanced learning capabilities and can adapt to new situations and environments. They are used in various fields, including robotics, autonomous vehicles, and healthcare.', 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': []}


In [5]:
from autogen import UserProxyAgent, ConversableAgent


chatbot = autogen.AssistantAgent(
    name="chatbot",
    system_message="For coding tasks, only use the functions you have been provided with. Reply TERMINATE when the task is done.",
    llm_config=local_llm_config,
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)


# define functions according to the function description


# one way of registering functions is to use the register_for_llm and register_for_execution decorators
@user_proxy.register_for_execution()
@chatbot.register_for_llm(name="python", description="run cell in ipython and return the execution result.")
def exec_python(cell: Annotated[str, "Valid Python cell to execute."]) -> str:
    ipython = get_ipython()
    result = ipython.run_cell(cell)
    log = str(result.result)
    if result.error_before_exec is not None:
        log += f"\n{result.error_before_exec}"
    if result.error_in_exec is not None:
        log += f"\n{result.error_in_exec}"
    return log


# another way of registering functions is to use the register_function
def exec_sh(script: Annotated[str, "Valid Python cell to execute."]) -> str:
    return user_proxy.execute_code_blocks([("sh", script)])


autogen.agentchat.register_function(
    exec_python,
    caller=chatbot,
    executor=user_proxy,
    name="sh",
    description="run a shell script and return the execution result.",
)

In [16]:
with Cache.disk() as cache:
    # start the conversation
    user_proxy.initiate_chat(
        chatbot,
        message="Can you give me a program to check the space in my system in python? Then execute it",
        cache=None,
        max_turns=4,
    )


user_proxy (to chatbot):

Can you give me a program to check the space in my system in python? Then execute it

--------------------------------------------------------------------------------
[autogen.oai.client: 12-05 13:26:38] {351} WARNING - Model granite30-8b is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
chatbot (to user_proxy):

To check the space in your system, you can use the `sh` function to run a shell command. Here's a simple Python program that uses the `df` command to display the amount of disk space used by the file system:

```python
{
    "type": "function",
    "function": {
        "name": "sh",
        "parameters": {
            "cell": "df -h"
        }
    }
}
```

This command will display a table with information about the disk space usage, including the total, used, and available space for each mounted file system.

Please note that the `sh` function is 

In [15]:
import os
import psutil

def check_memory():
    memory_info = psutil.virtual_memory()
    return {
        'total': memory_info.total,
        'available': memory_info.available,
        'percent': memory_info.percent
    }

check_memory()

{'total': 66351890432, 'available': 41663696896, 'percent': 37.2}